In [4]:
!pip install pymongo

In [5]:
from pymongo import MongoClient, InsertOne, UpdateOne, DeleteOne

# Establish client connection
client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']
students_collection = db['students']

# Clear collections (optional, supaya tidak dobel data)
students_collection.delete_many({})
courses_collection.delete_many({})

# Bulk insert for students
students_data = [
    InsertOne({"_id": 1, "name": "Alice", "major": "Computer Science"}),
    InsertOne({"_id": 2, "name": "Bob", "major": "Computer Science"}),
    InsertOne({"_id": 3, "name": "Charlie", "major": "Mathematics"}),
    InsertOne({"_id": 4, "name": "Dina", "major": "Physics"}),
    InsertOne({"_id": 5, "name": "Evan", "major": "Computer Science"}),
    InsertOne({"_id": 6, "name": "Farah", "major": "Computer Science"})
]

students_collection.bulk_write(students_data)
print("✅ Students inserted successfully.")

# Bulk insert for courses
courses_data = [
    InsertOne({"_id": "CS101", "title": "Introduction to Programming", "department": "Computer Science", "enrollments": [1, 2, 5]}),
    InsertOne({"_id": "CS201", "title": "Data Structures", "department": "Computer Science", "enrollments": list(range(1, 1+28))}),
    InsertOne({"_id": "MA101", "title": "Calculus I", "department": "Mathematics", "enrollments": [3]}),
    InsertOne({"_id": "PH101", "title": "General Physics", "department": "Physics", "enrollments": [4]}),
    InsertOne({"_id": "CS301", "title": "Operating Systems", "department": "Computer Science", "enrollments": [1, 2, 6]})
]

courses_collection.bulk_write(courses_data)
print("✅ Courses inserted successfully.")

print("\nSTUDENTS COLLECTION:")
for student in students_collection.find():
    print(student)

print("\nCOURSES COLLECTION:")
for course in courses_collection.find():
    print(course)

✅ Students inserted successfully.
✅ Courses inserted successfully.

STUDENTS COLLECTION:
{'_id': 1, 'name': 'Alice', 'major': 'Computer Science'}
{'_id': 2, 'name': 'Bob', 'major': 'Computer Science'}
{'_id': 3, 'name': 'Charlie', 'major': 'Mathematics'}
{'_id': 4, 'name': 'Dina', 'major': 'Physics'}
{'_id': 5, 'name': 'Evan', 'major': 'Computer Science'}
{'_id': 6, 'name': 'Farah', 'major': 'Computer Science'}

COURSES COLLECTION:
{'_id': 'CS101', 'title': 'Introduction to Programming', 'department': 'Computer Science', 'enrollments': [1, 2, 5]}
{'_id': 'CS201', 'title': 'Data Structures', 'department': 'Computer Science', 'enrollments': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]}
{'_id': 'MA101', 'title': 'Calculus I', 'department': 'Mathematics', 'enrollments': [3]}
{'_id': 'PH101', 'title': 'General Physics', 'department': 'Physics', 'enrollments': [4]}
{'_id': 'CS301', 'title': 'Operating Systems', 'department': 'Compute

In [6]:
pipeline = [
    {'$group': {'_id': '$department', 'course_count': {'$sum': 1}}},
    {'$sort': {'course_count': -1}}
]

for result in courses_collection.aggregate(pipeline):
    print(f"Department: {result['_id']}, Total Courses: {result['course_count']}")

Department: Computer Science, Total Courses: 3
Department: Mathematics, Total Courses: 1
Department: Physics, Total Courses: 1


In [7]:
pipeline = [
    #$match
    {'$match': {'department': 'Computer Science'}},{'$addFields': {'enrollment_count': { '$size': '$enrollments' }}},
    {'$match': {'enrollment_count': { '$gt': 25 }}},
    #$group
    {'$group': {'_id': '$department','total_courses': { '$sum': 1 },'courses': { '$push': '$title' },'total_enrollments': { '$sum': '$enrollment_count' }}}
]

for result in courses_collection.aggregate(pipeline):
    print(f"Department: {result['_id']}")
    print(f"Total Courses: {result['total_courses']}")
    print(f"Total Enrollment: {result['total_enrollments']}")
    print("Courses:", ", ".join(result['courses']))

print("\nCourses dengan Enrollments > 25 di Computer Science")
for course in courses_collection.aggregate([
    { '$match': { 'department': 'Computer Science' }},
    { '$addFields': { 'enrollment_count': { '$size': '$enrollments' }}},
    { '$match': { 'enrollment_count': { '$gt': 25 }}}
]):
    print(f"{course['title']}: {course['enrollment_count']} students")

Department: Computer Science
Total Courses: 1
Total Enrollment: 28
Courses: Data Structures

Courses dengan Enrollments > 25 di Computer Science
Data Structures: 28 students


In [17]:
pipeline_hw3 = [
        {'$lookup': {
            'from': 'students',
            'localField': 'enrollments', 
            'foreignField': '_id',        
            'as': 'enrolled_students'}},
        {'$project': {
            '_id': 1,
            'title': 1,
            'department': 1,
            'expected_enrollment_count': { '$size': '$enrollments' },
            'actual_student_count': { '$size': '$enrolled_students' },
            'enrolled_students': {
                '$map': {
                    'input': '$enrolled_students',
                    'as': 'student',
                    'in': {
                        'student_id': '$$student._id',
                        'name': '$$student.name',
                        'major': '$$student.major'}}}}}
]

print("Courses dengan detail mahasiswa yang telah terdaftar:")

for result in courses_collection.aggregate(pipeline_hw3):
    print(f"\n")
    print(f"Course ID: {result['_id']}")
    print(f"Title: {result['title']}")
    print(f"Department: {result['department']}")
    print(f"Expected Enrollment Count: {result['expected_enrollment_count']}")
    print(f"Actual Students Matched: {result['actual_student_count']}")

    if result['enrolled_students']:
        print("Students:")
        for student in result['enrolled_students']:
            print(f"  - {student['name']} (ID: {student['student_id']}, Major: {student['major']})")
    else:
        print("Students: No students found in system.")


Courses dengan detail mahasiswa yang telah terdaftar:


Course ID: CS101
Title: Introduction to Programming
Department: Computer Science
Expected Enrollment Count: 3
Actual Students Matched: 3
Students:
  - Alice (ID: 1, Major: Computer Science)
  - Bob (ID: 2, Major: Computer Science)
  - Evan (ID: 5, Major: Computer Science)


Course ID: CS201
Title: Data Structures
Department: Computer Science
Expected Enrollment Count: 28
Actual Students Matched: 6
Students:
  - Alice (ID: 1, Major: Computer Science)
  - Bob (ID: 2, Major: Computer Science)
  - Charlie (ID: 3, Major: Mathematics)
  - Dina (ID: 4, Major: Physics)
  - Evan (ID: 5, Major: Computer Science)
  - Farah (ID: 6, Major: Computer Science)


Course ID: MA101
Title: Calculus I
Department: Mathematics
Expected Enrollment Count: 1
Actual Students Matched: 1
Students:
  - Charlie (ID: 3, Major: Mathematics)


Course ID: PH101
Title: General Physics
Department: Physics
Expected Enrollment Count: 1
Actual Students Matched: 1
Studen